In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("book_cross_data.csv")

In [3]:
df.describe()

,Unnamed: 0,user_id,age,rating,year_of_publication
count,5.960220e+05,596022.000000,596022.000000,596022.000000,596022.000000
mean,4.943426e+05,142399.932893,36.480281,2.786885,1995.829989
std,2.928122e+05,79753.113338,10.410328,3.843312,6.772836
min,0.000000e+00,2.000000,5.000000,0.000000,1378.000000
25%,2.401482e+05,73694.000000,31.000000,0.000000,1993.000000
50%,4.912045e+05,143175.000000,34.743900,0.000000,1997.000000
75%,7.403948e+05,212898.000000,41.000000,7.000000,2001.000000
max,1.031173e+06,278854.000000,99.000000,10.000000,2008.000000


In [4]:
def sep_age(x) :
    if x < 10 :
        return '아동'
    elif 10 <= x < 20 :
        return '10대'
    elif 20 <= x < 30 :
        return '20대'
    elif 30 <= x < 40 :
        return '30대'
    elif 40 <= x < 50 :
        return '40대'
    elif 50 <= x < 60 :
        return '50대'
    elif 60 <= x < 70 :
        return '60대'
    elif 70 <= x < 80 :
        return '70대'
    elif 80 <= x < 90 :
        return '80대'
    else : 
        return '90대'

In [5]:
df['age_group'] = df['age'].apply(sep_age)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from ast import literal_eval
from sklearn.metrics.pairwise import linear_kernel

In [7]:
df['Information'] = df['Category'] + ' ' + df['Summary']

In [8]:
df['Information'] = df['Information'].str.lower()

In [9]:
import re

In [10]:
df['Information'] = [re.sub(r'[^a-z]+', ' ', x) for x in df['Information']]

In [12]:
book_df = df[['book_title', 'Category', 'Information', 'age_group']].drop_duplicates()

In [104]:
def get_rec_book(user_id) :
    info = df[df.user_id == user_id]
    
    user_books = info.book_title.unique()
    
    user_age = info.age_group.unique()[0]
    
    user_most_category = info.Category.value_counts().index[0]
    
    user_info = info[info.Category == user_most_category]
    
    user_sum_info = user_info['Information'].sum()
    
    find_books = book_df[(book_df.Category == user_most_category) & (book_df.age_group == user_age)]['Information']
    
    total_info = find_books.tolist() + [user_sum_info]
    
    tfidf = TfidfVectorizer(stop_words='english', lowercase= False, max_features = 2000)
    
    tfidf_matrix = tfidf.fit_transform(total_info)
    
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)[0][1:]
    
    rec10 = book_df[(book_df.Category == user_most_category) & (book_df.age_group == user_age)]
    
    rec10['cos_sim'] = cosine_sim
    
    final_rec10 = rec10.query('book_title not in @user_books').sort_values(by = 'cos_sim', ascending = False)[:5]
    
    
    return final_rec10

In [105]:
get_rec_book(278854)

<ipython-input-104-3ebdd95193f7>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rec10['cos_sim'] = cosine_sim


,book_title,Category,Information,age_group,cos_sim
567697,At Point Blank: A Suspense Novel,['Fiction'],fiction in a series as danger stalks point bl...,30대,1.000000
566902,All Men Are Brothers / Shui Hu Chuan,['Fiction'],fiction a band of men become robbers when the...,30대,1.000000
313239,Killer's Wake,['Fiction'],fiction despite the lure of the sea the heir ...,30대,0.774078
408839,The Vampire Papers,['Fiction'],fiction the sequel to i vampire finds the mal...,30대,0.593977
566209,Queer Fear: Gay Horror Fiction,['Fiction'],fiction a lambda award nominated anthology of...,30대,0.431756
